In [ ]:
# %% [markdown]
# # Model Comparison Notebook
# Цель: выбрать лучшую модель из унивариативного и мультивариативного набора,
# выполнить приближённый байесовский анализ (BIC → Bayes Factor), классические тесты,
# визуализации и краткий отчёт.

# %% [code]
# Блок 1: библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, log_loss, accuracy_score
import statsmodels.api as sm
from scipy.stats import ttest_rel, wilcoxon
import warnings
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')

df = pd.read_csv('forestfires.csv')
X = df.drop(columns=['area'])
y = df['area']

# Для демонстрации сгенерируем синтетические данные (регрессия)
from sklearn.datasets import make_regression, make_classification

X, y = make_regression(n_samples=500, n_features=8, n_informative=4, noise=10, random_state=0)
X = pd.DataFrame(X, columns=[f'x{i}' for i in range(X.shape[1])])
y = pd.Series(y, name='area')


print("X.shape:", X.shape, "y.shape:", y.shape)

# %% [markdown]
# Блок 3: Настройка CV и кандидатов моделей
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=0)

# Унивариативный кандидат: лучший отдельный признак (по CV metric)
# Мультивариативные кандидаты: линейная и случайный лес
multi_models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=0)
}

# %% [markdown]
# Блок 4: Функции для CV и метрик
def cv_metrics_regression(model, X, y, kf):
    metrics = []
    preds = []
    for train_idx, test_idx in kf.split(X):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        yhat = pd.Series(model.predict(X.iloc[test_idx]), index=test_idx)
        rmse = np.sqrt(mean_squared_error(y.iloc[test_idx], yhat))
        r2 = r2_score(y.iloc[test_idx], yhat)
        metrics.append({'rmse': rmse, 'r2': r2})
        preds.append((test_idx, yhat))
    return pd.DataFrame(metrics), preds

# Блок 5: Найдём лучший одиночный признак
results_uni = []
for col in X.columns:
    X_col = X[[col]]
    model = LinearRegression()
    df_metrics, _ = cv_metrics_regression(model, X_col, y, kf)
    results_uni.append((col, df_metrics['rmse'].mean(), df_metrics['rmse'].std(), df_metrics))

results_uni.sort(key=lambda x: x[1])   # сортируем по mean RMSE (меньше лучше)

best_uni = results_uni[0]
print("Best univariate feature:", best_uni[0], "mean_rmse:", best_uni[1], "std_rmse:", best_uni[2])

# Блок 6: CV для мультивариативных моделей
multi_metrics = {}
for name, model in multi_models.items():
    metrics_df, preds = cv_metrics_regression(model, X, y, kf)
    multi_metrics[name] = {'metrics': metrics_df, 'preds': preds}
    print(name, "mean metrics:\n", metrics_df.mean().to_dict())

# %% [markdown]
# Блок 7: Рассчитаем BIC / log-likelihood для параметрических моделей (statsmodels)
# Для регрессии: линейная модель через OLS (на всех данных)
bic_dict = {}

X_sm = sm.add_constant(X)  # включает константу
ols = sm.OLS(y, X_sm).fit()
bic_dict['OLS_full'] = {'bic': float(ols.bic), 'llf': float(ols.llf), 'k': int(ols.df_model + 1)}
# Для лучшего унивариативного признака:
col = best_uni[0]
X_sm_uni = sm.add_constant(X[[col]])
ols_uni = sm.OLS(y, X_sm_uni).fit()
bic_dict['OLS_uni'] = {'bic': float(ols_uni.bic), 'llf': float(ols_uni.llf), 'k': int(ols_uni.df_model + 1)}
print("BIC OLS full:", bic_dict['OLS_full'])
print("BIC OLS uni :", bic_dict['OLS_uni'])

# %% [markdown]
# Блок 8: Bayes Factor (приближённо через разницу BIC)
# BF_{1 over 2} ≈ exp((BIC2 - BIC1)/2)
def bayes_factor_from_bic(bic1, bic2):
    return np.exp((bic2 - bic1) / 2.0)

# Считаем BF между OLS_full и OLS_uni (если есть)
if ('OLS_full' in bic_dict) and ('OLS_uni' in bic_dict):
    bic1 = bic_dict['OLS_uni']['bic']
    bic2 = bic_dict['OLS_full']['bic']
    bf = bayes_factor_from_bic(bic1, bic2)  # BF uni over full
    print(f"BF (uni over full) ≈ {bf:.3g}")
    # интерпретация по Джеффриса
    def interpret_bf(bf):
        if bf < 1/100:
            return "Очень сильная поддержка модели 2 (Full)"
        if bf < 1/10:
            return "Сильная поддержка модели 2 (Full)"
        if bf < 1/3:
            return "Умеренная поддержка модели 2 (Full)"
        if bf < 1:
            return "Слабая поддержка модели 2 (Full)"
        if bf == 1:
            return "Нет различий"
        if bf <= 3:
            return "Слабая поддержка модели 1 (Uni)"
        if bf <= 10:
            return "Умеренная поддержка модели 1 (Uni)"
        if bf <= 30:
            return "Сильная поддержка модели 1 (Uni)"
        return "Очень сильная поддержка модели 1 (Uni)"
    print("Интерпретация:", interpret_bf(bf))

# %% [markdown]
# Блок 9: Статистические парные тесты по CV метрикам
# Сравним лучшую унивариативную модель и каждую мульти-модель по распределениям метрик
# Соберём векторы метрик (по fold) для сравнения
# лучший uni: получить его CV метрики (we stored df in best_uni[3])
uni_metrics_df = best_uni[3]  # DataFrame с K метриками
for name, info in multi_metrics.items():
    multi_df = info['metrics']
    # используем RMSE сравнение: uni rmse vs multi rmse
    try:
        uni_vec = uni_metrics_df['rmse'].values
        multi_vec = multi_df['rmse'].values
        # парный t-test
        t_stat, p_val = ttest_rel(uni_vec, multi_vec)
        # wilcoxon (если не нормальное)
        try:
            w_stat, w_p = wilcoxon(uni_vec, multi_vec)
        except Exception:
            w_stat, w_p = (np.nan, np.nan)
        print(f"Compare UNI({best_uni[0]}) vs {name}: t_p={p_val:.4f}, wilcoxon_p={w_p}")
    except Exception as e:
        print("Compare failed:", e)

# %% [markdown]
# Блок 10: Визуализации распределений метрик (boxplot / violin)
plt.figure(figsize=(10,6))
plot_df = []

for i,val in enumerate(uni_metrics_df['rmse']):
    plot_df.append({'model': f'UNI:{best_uni[0]}', 'fold': i, 'metric': val})
for name, info in multi_metrics.items():
    for i,val in enumerate(info['metrics']['rmse']):
        plot_df.append({'model': name, 'fold': i, 'metric': val})


plot_df = pd.DataFrame(plot_df)
sns.boxplot(x='model', y='metric', data=plot_df)
sns.swarmplot(x='model', y='metric', data=plot_df, color='0.25', alpha=0.8)
plt.title('Metric distributions by model (lower is better)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# %% [markdown]
# Блок 11: Таблица сравнения (mean ± std), BIC и BF где применимо
rows = []

rows.append({
    'model': f'UNI:{best_uni[0]}',
    'mean_metric': uni_metrics_df['rmse'].mean(),
    'std_metric': uni_metrics_df['rmse'].std(),
    'bic': bic_dict.get('OLS_uni', {}).get('bic', np.nan)
})

for name, info in multi_metrics.items():
    dfm = info['metrics']
    key = 'rmse'
    rows.append({
        'model': name,
        'mean_metric': dfm[key].mean(),
        'std_metric': dfm[key].std(),
        'bic': bic_dict.get('OLS_full', {}).get('bic', np.nan)
    })

comp_df = pd.DataFrame(rows).sort_values('mean_metric')
display(comp_df)

# %% [markdown]
# Блок 12: Краткий автоматический отчёт (строки, которые можно сохранить в файл)
report = []
report.append("INTRODUCTION")
report.append(f"Data shape: X={X.shape}, y={y.shape}")
report.append("\nPERFORMANCE SUMMARY")
report.append(comp_df.to_string(index=False))
# BF summary
if len(bic_dict) >= 2:
    report.append("\nBIC values:")
    for k,v in bic_dict.items():
        report.append(f"{k}: bic={v['bic']}, llf={v['llf']}")
    if 'OLS_uni' in bic_dict and 'OLS_full' in bic_dict:
        bf_val = bayes_factor_from_bic(bic_dict['OLS_uni']['bic'], bic_dict['OLS_full']['bic'])
        report.append(f"\nBayes Factor (uni over full) ≈ {bf_val:.3g}")
report.append("\nSTATISTICAL TESTS")
# add p-values from previous step (quick)
report.append("See printed p-values above (t-test and Wilcoxon) for pairwise comparisons between UNI and each multi model.")
report.append("\nCONCLUSION")
report.append("Choose model balancing predictive performance, statistical significance (p-values), and Bayes Factor (BIC based).")

print("\n\n".join(report))

X.shape: (500, 8) y.shape: (500,)


IndexError: list index out of range